In [15]:
# Imports
import random
import uuid
from datetime import datetime
import itertools

# Classe Nucleo - Armazena e manipula a sequência genética
class Nucleus():
    
    def __init__(self, karyotype = None):
        # O cariótipo de um indivíduo vai armazenar 4 pares de cromossomos 2n = 8.
        # Cada par é formado por um cromossomo vindo do pai e outro cromossomo vindo da mãe.
        # O cariótipo então é o conjunto dos pares. Precisa ter 8 elementos (4 pares de cromossomos).
        if karyotype == None:
            self.karyotype = []
        else:
            self.karyotype = karyotype
        self.nucleotides = [x for x in range(4)]
        
    def create_dna(self):
        # Método para criar DNA. Deve ser usado somente para a primeira geração de indivíduos.
        self.karyotype = [
            [x for x in random.choices(self.nucleotides, k=12)]
            for i in range(4)
        ]
    
    def __get_chromosome_pair(self, karyotype):
        # Método para agrupar os cromossomos homólogos.
        # Cromossomos homólogos = mesma posição, mesmo tamanho e com genes em posições correspondentes.
        self.chromosome_pairs = list(zip(karyotype[::2],karyotype[1::2]))
        return self.chromosome_pairs
    
    def __crossing_over(self, chromosome_pair, cutting_point):
        new_chromosome = [ x for x in chromosome_pair[0][0:cutting_point] + chromosome_pair[1][cutting_point:]]
        return new_chromosome
    
    def do_meiosis(self, cutting_points=None):
        # Método para executar a meiose, 1 célula diploide (2n = 8) tem que gerar 4 células haploides (n = 4).
        haploids = []
        
        # Trazendo a lista de cromossomos para a função
        pairs = self.__get_chromosome_pair(self.karyotype)
              
        from_father = []
        from_mother = []
        mixeds_1 = []
        mixeds_2 = []    
        
        if cutting_points == None:
            cutting_points = []
            for pair in pairs:
                cutting_points.append(random.randrange(len(pair[0])))
            
        index = 0

        
        for pair in pairs:
            
            from_father.append(pair[0])
            from_mother.append(pair[1])
            
            mixed1 = self.__crossing_over(pair, cutting_points[index])
            mixeds_1.append(mixed1)
            mixed2 = self.__crossing_over(pair[::-1], cutting_points[index])
            mixeds_2.append(mixed2)
            
            index += 1
        
        haploids.append({
            'origin':'father',
            'cutting_points':[],
            'chromosomes':from_father
        })
        haploids.append({
            'origin':'mother',
            'cutting_points':[],
            'chromosomes':from_mother
        })
        
        index = 0
        
        haploids.append({
            'origin':'mixed',
            'cutting_points':cutting_points, 
            'chromosomes':mixeds_1
        })
        
        haploids.append({
            'origin':'mixed',
            'cutting_points':cutting_points, 
            'chromosomes':mixeds_2
        })
            
        return haploids
    
    def to_dict(self):
        return {
            'karyotype':self.karyotype,
            'number_of_chromosomes':sum(len(pair['chromosomes']) for pair in self.karyotype),
            'mixed_count':sum(1 for pair in self.karyotype if pair['mixed']=='yes')
        }
    
# Criando uma classe que vai gerenciar a população
class PopulationManager():
    
    # Armazenando o contador e os IDs de todos os integrantes da população
    def __init__(self):
        self.population_counter = 0
        self.population = []
        
    # Método fertilization() que pede dois objetos Nucleus() e cria um indivíduo.
    def fertilization(self, father, mother):
        
        generation = 0
        
        generation_father = father['generation']
        generation_mother = mother['generation']
        
        generation = max(generation_father,generation_mother) + 1
        
        # Cria núcleos a partir de objetos
        nucleus_father = Nucleus(karyotype=father['karyotype'])
        nucleus_mother = Nucleus(karyotype=mother['karyotype'])
        
        # Executa a meiose dos dois objetos e armazena nas variáveis
        fat_gametes = nucleus_father.do_meiosis()
        mot_gametes = nucleus_mother.do_meiosis()

        # Escolhe um par de cromossomos por conjunto de gametas para formar um indivíduo
        fat_gamete = random.choice(fat_gametes)
        mot_gamete = random.choice(mot_gametes)

        # Cria o cariótipo do indivíduo novo
        #karyotype = list(zip(fat_gamete['chromosomes'],mot_gamete['chromosomes']))
        index = 0
        karyotype = []
        for i in fat_gamete['chromosomes']:
            karyotype.append(fat_gamete['chromosomes'][index])
            karyotype.append(mot_gamete['chromosomes'][index])
            index += 1
        
        # Retorna um dicionário com as informações do novo indivíduo
        new_individual = {
            'indv_id':str(uuid.uuid4()), 
            'father_id':father['indv_id'],
            'mother_id':mother['indv_id'],
            'generation':generation,
            'karyotype':karyotype,
            'chromosome_origin':{'father_gamete':fat_gamete['origin'],'mother_gamete':mot_gamete['origin']},
            'timestamp':datetime.now().isoformat(),
            'metadados':{
                'cutting_points':{
            'father': fat_gamete['cutting_points'],
            'mother': mot_gamete['cutting_points']
        },
                'method':'meiosis'
            }
        }

        self.population_counter += 1
        self.population.append(new_individual)
        
        return new_individual
        
        
    # Método para criar indivíduos que não vão ter pais
    def create_individual(self):
        
        # Criando uma instância da classe Nucleus()
        nucleus = Nucleus()
        nucleus.create_dna()
        
        # Geração 0
        generation = 0
        
        # Cria o dicionário do novo indivíduo
        new_individual = {
            'indv_id':str(uuid.uuid4()), 
            'father_id':None,
            'mother_id':None,
            'generation':generation,
            'karyotype':nucleus.karyotype,
            'chromosome_origin':None,
            'timestamp':datetime.now().isoformat(),
            'metadados':{
                'cutting_points':None,
                'method':'create_dna'
            }
        }
        
        self.population_counter += 1
        self.population.append(new_individual)
        
        return new_individual
    
# Classe indivíduo, que vai receber o DNA e exibir as características de acordo com os genes gerados.
class GeneReader():
    
    def __init__(self, individual_dictionary):
        self.karyotype = individual_dictionary['karyotype']
        self.color = None
        self.swimming_speed = 0
        self.gender = None
        
    
    def get_codons(self):
        codon_list = []
        for chromo in self.karyotype:
            new_list = []
            for i in range(0, len(chromo), 3):
                codon = chromo[i:i+3]
                if len(codon) == 3:
                    new_list.append(tuple(chromo[i:i+3]))
            codon_list.append(new_list)
        return codon_list

    def get_aminoacids(self, codon_list):
        aminoacids = []

        for chromo in codon_list:
            amino = []
            for codon in chromo:
                amino.append(inverted_aminoacid_table.get(codon, 'unknown'))
            aminoacids.append(amino)

        return aminoacids
    
    # Agora consigo analisar cada par de cromossomos.
    def get_color(self, aminoacids):
        
    # Vou definir arbitrariamente que o primeiro par define a cor.
    # A cor geralmente é definida pela quantidade de melanina, que é sintetizada pela tirosina (tyrosine)
        color_chromosomes = aminoacids[0] + aminoacids[1]
        color_gene = {x: color_chromosomes.count(x) for x in set(color_chromosomes)}
        self.color = hex(color_gene['tyrosine']*1000)
        
        return self.color
    
    def get_swimming_speed(self, aminoacids):
    # Definindo arbitrariamente que a velocidade de nado está associada com o segundo par de cromossomos.
        swim_chromosomes = aminoacids[2] + aminoacids[3]
        swim_gene = {x: swim_chromosomes.count(x) for x in set(swim_chromosomes)}
        self.swimming_speed = (swim_gene['arginine'] * 5) - swim_gene['stop']
    
        return self.swimming_speed

# 3 Nucleotídeos == 1 Códon. 
# 1 códon gera aminoácidos. 
# Aminoácidos unidos geram proteínas. 
# Proteínas definem as características do ser.

# Criando uma tabela com as possíveis combinações de nucleotídeos que vão gerar códons diferentes.
codon_table = [x for x in itertools.product([0,1,2,3],repeat=3)]

# Tabela de códons RNA.
# 0 = Adenosina
# 1 = Guanina
# 2 = Citosina
# 3 = Timina

aminoacid_table = {
    'alanine':[(1,2,3),(1,2,2),(1,2,0),(1,2,1)],
    'arginine':[(2,1,3),(2,1,2),(2,1,0),(2,1,1),(0,1,0),(0,1,1)],
    'asparagine':[(0,0,3),(0,0,2)],
    'aspartic_acid':[(1,0,3),(1,0,2)],
    'cysteine':[(3,1,3),(3,1,2)],
    'glutamine':[(2,0,0),(2,0,1)],
    'glutamic_acid':[(1,0,0),(1,0,1)],
    'glycine':[(1,1,3),(1,1,2),(1,1,0),(1,1,1)],
    'histidine':[(2,0,3),(2,0,2)],
    'start':[(0,3,1),(2,3,1),(3,3,1)],
    'isoleucine':[(0,3,3),(0,3,2),(0,3,0)],
    'leucine':[(2,3,3),(2,3,2),(2,3,0),(2,3,1),(3,3,0),(3,3,1)],
    'lysine':[(0,0,0),(0,0,1)],
    'phenylalanine':[(3,3,3),(3,3,2)],
    'proline':[(2,2,3),(2,2,2),(2,2,0),(2,2,1)],
    'serine':[(3,2,3),(3,2,2),(3,2,0),(3,2,1),(0,1,3),(0,1,2)],
    'threonine':[(0,2,3),(0,2,2),(0,2,0),(0,2,1)],
    'tryptophan':[(3,1,1)],
    'tyrosine':[(3,0,3),(3,0,2)],
    'valine':[(1,3,3),(1,3,2),(1,3,0),(1,3,1)],
    'stop':[(3,0,0),(3,1,0),(3,0,1)]
}

inverted_aminoacid_table = {}

for key in aminoacid_table.keys():
    for codon in aminoacid_table[key]:
        inverted_aminoacid_table.update({codon:key})

In [16]:
eden = PopulationManager()

In [17]:
adam = eden.create_individual()

In [18]:
eve = eden.create_individual()

In [19]:
adam

{'indv_id': 'ccdefa55-d01c-429c-aad7-17c69cf5acf1',
 'father_id': None,
 'mother_id': None,
 'generation': 0,
 'karyotype': [[3, 3, 0, 0, 2, 0, 0, 0, 3, 3, 3, 1],
  [0, 0, 0, 1, 1, 0, 1, 1, 3, 3, 0, 3],
  [0, 1, 0, 2, 2, 0, 3, 0, 1, 0, 1, 0],
  [2, 1, 1, 1, 3, 1, 2, 2, 0, 1, 0, 3]],
 'chromosome_origin': None,
 'timestamp': '2025-06-23T18:46:53.689878',
 'metadados': {'cutting_points': None, 'method': 'create_dna'}}

In [20]:
eve

{'indv_id': 'd8071835-8bb9-4634-866b-3aa199ee8b8a',
 'father_id': None,
 'mother_id': None,
 'generation': 0,
 'karyotype': [[1, 3, 2, 3, 1, 2, 3, 2, 1, 2, 3, 2],
  [3, 3, 1, 2, 2, 3, 0, 2, 1, 0, 1, 0],
  [3, 3, 1, 3, 1, 1, 2, 3, 0, 3, 1, 0],
  [1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 3]],
 'chromosome_origin': None,
 'timestamp': '2025-06-23T18:46:54.739405',
 'metadados': {'cutting_points': None, 'method': 'create_dna'}}

In [21]:
nucleo = Nucleus()

In [22]:
nucleo.karyotype = adam['karyotype']

In [23]:
nucleo.do_meiosis()

[{'origin': 'father',
  'cutting_points': [],
  'chromosomes': [[3, 3, 0, 0, 2, 0, 0, 0, 3, 3, 3, 1],
   [0, 1, 0, 2, 2, 0, 3, 0, 1, 0, 1, 0]]},
 {'origin': 'mother',
  'cutting_points': [],
  'chromosomes': [[0, 0, 0, 1, 1, 0, 1, 1, 3, 3, 0, 3],
   [2, 1, 1, 1, 3, 1, 2, 2, 0, 1, 0, 3]]},
 {'origin': 'mixed',
  'cutting_points': [2, 9],
  'chromosomes': [[3, 3, 0, 1, 1, 0, 1, 1, 3, 3, 0, 3],
   [0, 1, 0, 2, 2, 0, 3, 0, 1, 1, 0, 3]]},
 {'origin': 'mixed',
  'cutting_points': [2, 9],
  'chromosomes': [[0, 0, 0, 0, 2, 0, 0, 0, 3, 3, 3, 1],
   [2, 1, 1, 1, 3, 1, 2, 2, 0, 0, 1, 0]]}]

In [24]:
nucleo2 = Nucleus()

In [25]:
nucleo2.karyotype = eve['karyotype']

In [26]:
nucleo2.karyotype

[[1, 3, 2, 3, 1, 2, 3, 2, 1, 2, 3, 2],
 [3, 3, 1, 2, 2, 3, 0, 2, 1, 0, 1, 0],
 [3, 3, 1, 3, 1, 1, 2, 3, 0, 3, 1, 0],
 [1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 3]]

In [27]:
nucleo2.do_meiosis()

[{'origin': 'father',
  'cutting_points': [],
  'chromosomes': [[1, 3, 2, 3, 1, 2, 3, 2, 1, 2, 3, 2],
   [3, 3, 1, 3, 1, 1, 2, 3, 0, 3, 1, 0]]},
 {'origin': 'mother',
  'cutting_points': [],
  'chromosomes': [[3, 3, 1, 2, 2, 3, 0, 2, 1, 0, 1, 0],
   [1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 3]]},
 {'origin': 'mixed',
  'cutting_points': [1, 8],
  'chromosomes': [[1, 3, 1, 2, 2, 3, 0, 2, 1, 0, 1, 0],
   [3, 3, 1, 3, 1, 1, 2, 3, 1, 0, 1, 3]]},
 {'origin': 'mixed',
  'cutting_points': [1, 8],
  'chromosomes': [[3, 3, 2, 3, 1, 2, 3, 2, 1, 2, 3, 2],
   [1, 0, 1, 1, 1, 0, 2, 0, 0, 3, 1, 0]]}]

In [28]:
caim = eden.fertilization(adam,eve)

In [29]:
caim

{'indv_id': 'f9f886a8-3e90-43f3-8035-faae0cf35f25',
 'father_id': 'ccdefa55-d01c-429c-aad7-17c69cf5acf1',
 'mother_id': 'd8071835-8bb9-4634-866b-3aa199ee8b8a',
 'generation': 1,
 'karyotype': [[3, 3, 0, 0, 2, 0, 0, 0, 3, 3, 3, 1],
  [3, 3, 2, 3, 1, 2, 3, 2, 1, 2, 3, 2],
  [0, 1, 0, 2, 2, 0, 3, 0, 1, 0, 1, 0],
  [1, 0, 1, 3, 1, 1, 2, 3, 0, 3, 1, 0]],
 'chromosome_origin': {'father_gamete': 'father', 'mother_gamete': 'mixed'},
 'timestamp': '2025-06-23T18:46:58.937088',
 'metadados': {'cutting_points': {'father': [], 'mother': [2, 3]},
  'method': 'meiosis'}}

In [30]:
lab = GeneReader(caim)

In [31]:
lab.get_codons()

[[(3, 3, 0), (0, 2, 0), (0, 0, 3), (3, 3, 1)],
 [(3, 3, 2), (3, 1, 2), (3, 2, 1), (2, 3, 2)],
 [(0, 1, 0), (2, 2, 0), (3, 0, 1), (0, 1, 0)],
 [(1, 0, 1), (3, 1, 1), (2, 3, 0), (3, 1, 0)]]

In [32]:
teste = [[(1, 1, 0), (3, 0, 3), (1, 1, 2), (3, 1, 2)],
 [(3, 2, 0), (3, 2, 2), (3, 3, 1), (0, 3, 2)],
 [(1, 1, 2), (0, 2, 3), (0, 0, 3), (1, 1, 2)],
 [(0, 2, 3), (3, 0, 2), (3, 2, 1), (1, 2, 1)]]

In [33]:
# Definir aminoácidos
# Agrupar de dois em dois

aminoacids = []

for chromo in teste:
    amino = []
    for codon in chromo:
        amino.append(inverted_aminoacid_table[codon])
    aminoacids.append(amino)

aminoacids

[['glycine', 'tyrosine', 'glycine', 'cysteine'],
 ['serine', 'serine', 'leucine', 'isoleucine'],
 ['glycine', 'threonine', 'asparagine', 'glycine'],
 ['threonine', 'tyrosine', 'serine', 'alanine']]

In [34]:
opa = [['glycine', 'tyrosine', 'glycine', 'cysteine'],
 ['serine', 'serine', 'leucine', 'isoleucine'],
 ['glycine', 'threonine', 'asparagine', 'glycine'],
 ['threonine', 'tyrosine', 'serine', 'alanine']]

In [35]:
opa2 = opa[0] + opa[1]
color_gene = {x: opa2.count(x) for x in set(opa2)}

In [36]:
color_gene

{'tyrosine': 1,
 'cysteine': 1,
 'leucine': 1,
 'isoleucine': 1,
 'glycine': 2,
 'serine': 2}

In [37]:
caim

{'indv_id': 'f9f886a8-3e90-43f3-8035-faae0cf35f25',
 'father_id': 'ccdefa55-d01c-429c-aad7-17c69cf5acf1',
 'mother_id': 'd8071835-8bb9-4634-866b-3aa199ee8b8a',
 'generation': 1,
 'karyotype': [[3, 3, 0, 0, 2, 0, 0, 0, 3, 3, 3, 1],
  [3, 3, 2, 3, 1, 2, 3, 2, 1, 2, 3, 2],
  [0, 1, 0, 2, 2, 0, 3, 0, 1, 0, 1, 0],
  [1, 0, 1, 3, 1, 1, 2, 3, 0, 3, 1, 0]],
 'chromosome_origin': {'father_gamete': 'father', 'mother_gamete': 'mixed'},
 'timestamp': '2025-06-23T18:46:58.937088',
 'metadados': {'cutting_points': {'father': [], 'mother': [2, 3]},
  'method': 'meiosis'}}

In [39]:
caim['metadados']['cutting_points']

{'father': [], 'mother': [2, 3]}